### 모듈 불러오기

In [1]:
!pip install pillow   
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os, glob

### 데이터 불러오기

In [2]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    
    number_of_data = 0 
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        number_of_data+=1
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        number_of_data+=1
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        number_of_data+=1
    # 위 반복문을 실행하면 전체 이미지 파일 개수를 알 수 있다. 
    
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("이미지 개수는",idx,"입니다.")
    return imgs, labels

--------------------------------------------------------------------------------------------------------------

In [3]:
image_dir_path = os.getenv("HOME") + "/aiffel/1.train/train_dataset_total/*"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
print(image_dir_path)
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

이미지 개수는 15000 입니다.
/home/aiffel/aiffel/1.train/train_dataset_total/*
x_train shape: (15000, 28, 28, 3)
y_train shape: (15000,)


### 딥러닝 네트워크 설계하기

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]
#바꿔 볼 수 있는 하이퍼파라미터들
n_channel_1=16
n_channel_2=32
n_dense=32
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

### 딥러닝 네트워크 학습시키기

In [5]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
# [[YOUR CODE]]
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=n_train_epoch)

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 1.0473 - accuracy: 0.4192
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.7057 - accuracy: 0.6897
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.5086 - accuracy: 0.8044
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3924 - accuracy: 0.8495
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3020 - accuracy: 0.8892
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2438 - accuracy: 0.9107
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1794 - accuracy: 0.9371
Epoch 8/10
469/469 [==============================] - 1s 3ms/step - loss: 0.1650 - accuracy: 0.9455
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1273 - accuracy: 0.9565
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0976 - accuracy: 0.9698

### 얼마나 잘 만들었는지 확인하기(테스트)

#### 테스트용 데이터인 x_test, y_test를 만들어 봅시다

In [8]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
# [[YOUR CODE]]
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_wbtest/"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


#### 위에서 훈련시킨 model을 사용하여 test_accuracy를 측정해 봅시다

In [9]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.evaluate()을 사용해 봅시다.
# [[YOUR CODE]]
test_loss, test_accuracy = model.evaluate(x_test_norm,  y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 1.0752 - accuracy: 0.6367
test_loss: 1.0751582384109497 
test_accuracy: 0.6366666555404663


### 느낀점(회고)

trained 데이터의 accuracy(0.9698)와 비교하여 test 데이터의 accuracy가 0.6367로 낮게 나왔다.
test 데이터의 loss 값이 1.075로 trained 데이터 loss값(0.097)보다 높게 나왔다. 
설계한 모델이 trained 데이터에 fit한 경향이 있음을 보여주고 있다.